In [1]:
#train model from panel prepared in previous module
#start with naive model for each asset

In [2]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import TimeSeriesSplit

In [3]:
pd.set_option("display.max_columns", 500)
pd.set_option("display.max_rows", 500)

In [4]:
data_folder  = "../DataWork/"
panel_fname  = data_folder + "panel.csv"  #panel features file to load
target_fname = data_folder + "target.csv" #target file to load

In [5]:
#work with multiindex to allow selection of subset of features based on stock before flattening - TBD
dat = pd.read_csv(panel_fname, header=[0,1], index_col=0)
dat.index = pd.to_datetime(dat.index, format = "%Y-%m-%d")
dat.index.name = 'Date'
dat.tail()

Features       Ret20                                                    \
Stocks          ABBV       ACN       AEP       AIZ      ALLE      AMAT   
Date                                                                     
2022-02-14       NaN       NaN       NaN       NaN       NaN       NaN   
2022-02-15  0.056703 -0.055704 -0.046859  0.081916 -0.056451 -0.085747   
2022-02-16  0.075887 -0.029583 -0.046989  0.098407 -0.064598 -0.014858   
2022-02-17  0.085952 -0.059453 -0.041836  0.107424 -0.052362 -0.019448   
2022-02-18  0.087371 -0.044008 -0.039822  0.118848 -0.060764 -0.012742   

Features                                                                \
Stocks           AMP      AMZN       AVB       AVY       AXP       BDX   
Date                                                                     
2022-02-14       NaN       NaN       NaN       NaN       NaN       NaN   
2022-02-15 -0.030582 -0.015262 -0.032876 -0.114062  0.175155  0.040376   
2022-02-16  0.004444  0.011460 -0.025112 -0.110519  0.206361  0.027142   
2022-02-17 -0.029308  0.019490 -0.030158 -0.105656  0.195727  0.008225   
2022-02-18  0.007481  0.067485 -0.028969 -0.104332  0.205116 -0.002232   

Features                                                                \
Stocks          BF-B       BMY        BR      CARR       CDW        CE   
Date                                                                     
2022-02-14       NaN       NaN       NaN       NaN       NaN       NaN   
2022-02-15 -0.001203  0.043426 -0.079965 -0.081807 -0.033840 -0.074298   
2022-02-16 -0.002549  0.043538 -0.094768 -0.066522 -0.020914 -0.074126   
2022-02-17 -0.004687  0.042592 -0.115361 -0.074540 -0.036097 -0.075905   
2022-02-18  0.011187  0.061198 -0.095195 -0.081019 -0.015829 -0.103224   

Features                                                                \
Stocks          CHTR       CNC       CNP       COP      CTAS       CZR   
Date                                                                     
2022-02-14       NaN       NaN       NaN       NaN       NaN       NaN   
2022-02-15  0.044799  0.066253 -0.016596  0.029512 -0.032095  0.061486   
2022-02-16  0.047306  0.081736 -0.010992  0.038036 -0.021659  0.093439   
2022-02-17  0.043888  0.047734 -0.008468  0.067598 -0.023196  0.062197   
2022-02-18  0.046540  0.061829 -0.020722  0.085246 -0.018611  0.093961   

Features                                                                \
Stocks            DG       DPZ       DRE       DXC       EWA       EWC   
Date                                                                     
2022-02-14       NaN       NaN       NaN       NaN       NaN       NaN   
2022-02-15 -0.067750 -0.076747 -0.084424  0.137209 -0.003727 -0.005135   
2022-02-16 -0.108376 -0.079911 -0.071478  0.154285  0.002466 -0.005922   
2022-02-17 -0.073245 -0.049262 -0.081189  0.110996  0.002071 -0.010414   
2022-02-18 -0.049797  0.001861 -0.080116  0.153870  0.022776  0.003981   

Features                                                                \
Stocks           EWG       EWH       EWJ       EWL       EWQ       EWT   
Date                                                                     
2022-02-14       NaN       NaN       NaN       NaN       NaN       NaN   
2022-02-15 -0.016647  0.037442 -0.022930 -0.036324 -0.019291 -0.014352   
2022-02-16 -0.018451  0.034671 -0.010305 -0.031234 -0.021037 -0.005271   
2022-02-17 -0.032065  0.007792 -0.020654 -0.040731 -0.026528 -0.011423   
2022-02-18 -0.028749  0.000000 -0.018375 -0.031386 -0.014031  0.012451   

Features                                                                \
Stocks           EWU       EWY       EWZ        FB       FTV      GOOG   
Date                                                                     
2022-02-14       NaN       NaN       NaN       NaN       NaN       NaN   
2022-02-15  0.002323 -0.029639  0.142362 -0.364360 -0.102220  0.000990   
2022-02-16  0.002311 -0.028763  0.118192 -0.389264 -0.100083  0.013440  

In [6]:
tgt = pd.read_csv(target_fname, header=[0,1], index_col=0)
tgt.index = pd.to_datetime(tgt.index, format = "%Y-%m-%d")
tgt.index.name = 'Date'
tgt.head()

Features   Target                                                             \
Stocks       ABBV       ACN       AEP       AIZ ALLE      AMAT AMP      AMZN   
Date                                                                           
2004-12-31    NaN -0.035819  0.026155  0.062798  NaN -0.072759 NaN -0.024456   
2005-01-03    NaN -0.014515  0.045959  0.071942  NaN -0.025287 NaN -0.046905   
2005-01-04    NaN  0.003102  0.050629  0.096216  NaN -0.014652 NaN -0.006189   
2005-01-05    NaN -0.006650  0.061661  0.104254  NaN -0.003706 NaN -0.155628   
2005-01-06    NaN  0.012121  0.062204  0.092931  NaN  0.041964 NaN -0.139080   

Features                                                                    \
Stocks           AVB       AVY       AXP       BDX      BF-B       BMY  BR   
Date                                                                         
2004-12-31 -0.117983  0.001999 -0.052882 -0.002644 -0.009287 -0.077734 NaN   
2005-01-03 -0.091420  0.012095  0.017093  0.027992  0.008290 -0.060948 NaN   
2005-01-04 -0.066307  0.032872  0.014813  0.070773  0.019285 -0.035608 NaN   
2005-01-05 -0.019177  0.033612  0.017661  0.076557  0.029005 -0.039076 NaN   
2005-01-06 -0.026057  0.037854  0.030190  0.073848  0.044442 -0.025783 NaN   

Features                                                                      \
Stocks     CARR CDW  CE CHTR       CNC       CNP       COP      CTAS CZR  DG   
Date                                                                           
2004-12-31  NaN NaN NaN  NaN  0.168410 -0.004435  0.066387 -0.008242 NaN NaN   
2005-01-03  NaN NaN NaN  NaN  0.151806  0.034654  0.117638  0.017000 NaN NaN   
2005-01-04  NaN NaN NaN  NaN  0.142983  0.061108  0.129153  0.054871 NaN NaN   
2005-01-05  NaN NaN NaN  NaN  0.182082  0.084683  0.129827  0.054786 NaN NaN   
2005-01-06  NaN NaN NaN  NaN  0.191211  0.101543  0.119291  0.051890 NaN NaN   

Features                                                                \
Stocks           DPZ       DRE       DXC       EWA       EWC       EWG   
Date                                                                     
2004-12-31 -0.064988 -0.093262 -0.089967  0.007755 -0.046767 -0.048387   
2005-01-03 -0.060552 -0.075391 -0.072946  0.031084 -0.027237 -0.030654   
2005-01-04 -0.052490 -0.045946 -0.060831  0.061715  0.006597 -0.007177   
2005-01-05 -0.048216 -0.007214 -0.057007  0.045147  0.017529 -0.016071   
2005-01-06 -0.014011  0.017656 -0.032988  0.053841  0.021233 -0.008276   

Features                                                                \
Stocks           EWH       EWJ       EWL       EWQ       EWT       EWU   
Date                                                                     
2004-12-31 -0.048295 -0.030686 -0.037919 -0.015615 -0.023491 -0.024624   
2005-01-03 -0.040823 -0.013060 -0.035528 -0.013881 -0.007611 -0.006696   
2005-01-04 -0.018966  0.007512 -0.003610  0.013161  0.031398  0.019080   
2005-01-05 -0.002623 -0.005677 -0.010263  0.004259  0.023953  0.015173   
2005-01-06  0.005177 -0.001889 -0.008434  0.004696  0.045732  0.021301   

Features                                                                  \
Stocks           EWY       EWZ  FB FTV      GOOG       GPC GSG       HIG   
Date                                                                       
2004-12-31  0.057139 -0.031981 NaN NaN  0.014573 -0.040056 NaN -0.029578   
2005-01-03  0.068646 -0.004164 NaN NaN -0.054802 -0.021317 NaN -0.000290   
2005-01-04  0.090569  0.054448 NaN NaN  0.057250  0.004847 NaN  0.011399   
2005-01-05  0.085447  0.073169 NaN NaN  0.085865  0.025183 NaN  0.022765   
2005-01-06  0.122491  0.099452 NaN NaN  0.080520  0.040934 NaN  0.045270   

Features                                                                    \
Stocks     HIGH.L       HST HYG IAU ICLN IEAA.L       IEF IEFM.L IEMG IEUS   
Date                                                                         
2004-12-31    NaN -0.078118 NaN NaN  NaN    NaN  0.008674    NaN

In [7]:
tgt.shape

(4384, 100)

In [8]:
features = ["_".join(reversed(a)) for a in dat.columns.to_flat_index()]

In [9]:
def get_Xy(df, tgt):
    """return data filtered for rows with valid target"""
    X = np.array(df.loc[tgt.dropna().index])
    y = tgt.dropna().values
    return X, y

In [10]:
def train_stock(asset = 0):
    X, y = get_Xy(dat,tgt.iloc[:,asset])
    preds = np.zeros(len(y)) #to hold predictions
    cv = TimeSeriesSplit(n_splits=5, max_train_size = len(y)//5,gap = 21)
    fold_scores = []
    models = []
    feat_imp =[]
    for i, (train_idx, val_idx) in enumerate(cv.split(X)):
        lgbm = lgb.LGBMRegressor()
        X_train, X_val = X[train_idx,:], X[val_idx,:]
        y_train, y_val = y[train_idx], y[val_idx]
        print(f'Fold {i+1:2} Train length: {len(train_idx)} Test length: {len(val_idx)}')
        lgbm = lgbm.fit(X_train,y_train, eval_set = [(X_val, y_val)], 
                        callbacks = [lgb.early_stopping(10), lgb.log_evaluation(100)])
        pred_val = lgbm.predict(X_val)
        models.append(lgbm)
        fold_scores.append(lgbm.best_score_['valid_0']['l2'])
        feat_imp.append(lgbm.feature_importances_)
        preds[val_idx] += pred_val
    avg_feat_imp = np.mean(feat_imp,axis=0)
    return preds, models, avg_feat_imp, fold_scores

In [11]:
asset_preds = {}
asset_models = {}
asset_feat_imp = {}
asset_fold_scores = {}

In [12]:
%%time
for i in range(tgt.shape[1]):
    print(f"***TRAIN asset {i+1} {tgt.columns.levels[1][i]}***")
    asset_preds[i], asset_models[i], asset_feat_imp[i], asset_fold_scores[i] = train_stock(i)

***TRAIN asset 1 ABBV***
Fold  1 Train length: 355 Test length: 373
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.0063883
Fold  2 Train length: 448 Test length: 373
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[3]	valid_0's l2: 0.00170398
Fold  3 Train length: 448 Test length: 373
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[17]	valid_0's l2: 0.00801738
Fold  4 Train length: 448 Test length: 373
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.00738119
Fold  5 Train length: 448 Test length: 373
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.00431142
***TRAIN asset 2 ACN***
Fold  1 Train length: 688 Test length: 705
Training until validation scores don't improve for 10 rounds
Early s

Early stopping, best iteration is:
[7]	valid_0's l2: 0.00307343
Fold  5 Train length: 846 Test length: 705
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.00638174
***TRAIN asset 11 AXP***
Fold  1 Train length: 688 Test length: 705
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.0258432
Fold  2 Train length: 846 Test length: 705
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.00391693
Fold  3 Train length: 846 Test length: 705
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.00388799
Fold  4 Train length: 846 Test length: 705
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[5]	valid_0's l2: 0.00231003
Fold  5 Train length: 846 Test length: 705
Training until validation sco

Early stopping, best iteration is:
[1]	valid_0's l2: 0.00663478
Fold  4 Train length: 846 Test length: 705
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.00546349
Fold  5 Train length: 846 Test length: 705
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.00871038
***TRAIN asset 21 CNP***
Fold  1 Train length: 688 Test length: 705
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.00983008
Fold  2 Train length: 846 Test length: 705
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[2]	valid_0's l2: 0.0021962
Fold  3 Train length: 846 Test length: 705
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.00239394
Fold  4 Train length: 846 Test length: 705
Training until validation sco

Early stopping, best iteration is:
[1]	valid_0's l2: 0.00247001
Fold  3 Train length: 846 Test length: 705
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.00221695
Fold  4 Train length: 846 Test length: 705
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.00121678
Fold  5 Train length: 846 Test length: 705
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.00545831
***TRAIN asset 31 EWG***
Fold  1 Train length: 688 Test length: 705
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.0108803
Fold  2 Train length: 846 Test length: 705
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.00562197
Fold  3 Train length: 846 Test length: 705
Training until validation sco

Fold  1 Train length: 379 Test length: 398
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.00519481
Fold  2 Train length: 478 Test length: 398
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.00309089
Fold  3 Train length: 478 Test length: 398
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.00529515
Fold  4 Train length: 478 Test length: 398
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[2]	valid_0's l2: 0.0100186
Fold  5 Train length: 478 Test length: 398
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.00901798
***TRAIN asset 41 FTV***
Fold  1 Train length: 211 Test length: 228
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration i

...

Early stopping, best iteration is:
[1]	valid_0's l2: 0.00261933
***TRAIN asset 60 IUVL.L***
Fold  1 Train length: 200 Test length: 218
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.000931685
Fold  2 Train length: 262 Test length: 218
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.00250491
Fold  3 Train length: 262 Test length: 218
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.012271
Fold  4 Train length: 262 Test length: 218
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[2]	valid_0's l2: 0.00504655
Fold  5 Train length: 262 Test length: 218
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.000977693
***TRAIN asset 61 IVV***
Fold  1 Train length: 688 Test length: 705


Early stopping, best iteration is:
[1]	valid_0's l2: 0.000789358
***TRAIN asset 70 OGN***
Fold  1 Train length: 9 Test length: 28
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.220748
Fold  2 Train length: 34 Test length: 28
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.0260089
Fold  3 Train length: 34 Test length: 28
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.0247334
Fold  4 Train length: 34 Test length: 28
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.0209
Fold  5 Train length: 34 Test length: 28
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.0126807
***TRAIN asset 71 PG***
Fold  1 Train length: 688 Test length: 705
Training until validati

Fold  4 Train length: 630 Test length: 525
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.000114121
Fold  5 Train length: 630 Test length: 525
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[8]	valid_0's l2: 0.000153061
***TRAIN asset 80 SHY***
Fold  1 Train length: 688 Test length: 705
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 2.79434e-05
Fold  2 Train length: 846 Test length: 705
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[18]	valid_0's l2: 7.73259e-06
Fold  3 Train length: 846 Test length: 705
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[9]	valid_0's l2: 4.08433e-06
Fold  4 Train length: 846 Test length: 705
Training until validation scores don't improve for 10 rounds
Early stopping, best iter

Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.00846863
Fold  3 Train length: 325 Test length: 271
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[4]	valid_0's l2: 0.0058417
Fold  4 Train length: 325 Test length: 271
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[2]	valid_0's l2: 0.0226946
Fold  5 Train length: 325 Test length: 271
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.00615079
***TRAIN asset 90 XLB***
Fold  1 Train length: 688 Test length: 705
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.00980301
Fold  2 Train length: 846 Test length: 705
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.00322309
Fold  3 Trai

Early stopping, best iteration is:
[1]	valid_0's l2: 0.00254458
***TRAIN asset 99 XLY***
Fold  1 Train length: 688 Test length: 705
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.00844913
Fold  2 Train length: 846 Test length: 705
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[2]	valid_0's l2: 0.00213529
Fold  3 Train length: 846 Test length: 705
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid_0's l2: 0.00140476
Fold  4 Train length: 846 Test length: 705
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[5]	valid_0's l2: 0.00108826
Fold  5 Train length: 846 Test length: 705
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[4]	valid_0's l2: 0.00483903
***TRAIN asset 100 XOM***
Fold  1 Train length: 688 Test length: 705
Tr

In [13]:
asset_fold_scores[99]

[0.004258192432534523,
 0.0024604312685327146,
 0.0021904796752597842,
 0.002164406102723723,
 0.013253943155827456]

In [14]:
eval_scores = np.zeros(tgt.shape[1])
eval_std    = np.zeros(tgt.shape[1])
for i in range(tgt.shape[1]):
    eval_scores[i] = np.mean(asset_fold_scores[i])*100
    eval_std[i]    = np.std(asset_fold_scores[i]) *100

In [15]:
eval_score_df = pd.DataFrame({'Mean Score': eval_scores, 'Stdev' : eval_std}, index = dat.columns.levels[1])
eval_score_df

,Mean Score,Stdev
Stocks,,
ABBV,0.556045,0.230084
ACN,0.381381,0.147664
AEP,0.283660,0.140227
AIZ,0.970572,1.075435
ALLE,0.432798,0.304277
AMAT,0.887598,0.228303
AMP,1.199116,0.916370
AMZN,0.827696,0.478562
AVB,0.570563,0.501052


In [16]:
eval_score_df.describe()

,Mean Score,Stdev
count,100.000000,100.000000
mean,0.712520,0.627134
std,0.995424,1.298444
min,0.001193,0.000883
25%,0.292320,0.159023
50%,0.444600,0.272935
75%,0.731985,0.501378
max,6.101422,8.000231


In [17]:
features_to_predict = dat.iloc[-1,:].copy().values.reshape(1,-1) #temporary, use last row of data for prediction
preds = np.zeros(tgt.shape[1])
for i in range(tgt.shape[1]):
    models = asset_models[i]
    preds[i] = np.mean(np.array([model.predict(features_to_predict) for model in models]),axis=0)

In [18]:
pd.DataFrame(preds).describe()

,0
count,100.000000
mean,0.007501
std,0.012269
min,-0.040782
25%,0.002551
50%,0.006938
75%,0.011027
max,0.077869


In [19]:
pd.DataFrame(preds).to_csv(data_folder + "return_predictions.csv",index=False,header=False)